In [17]:
import requests
import json
import typing
import re

class netBoxObject():
    def __init__(self, url: str = 'https://netbox.mda.mil', api_key: str = None):
        self.url = url
        self.key = api_key

class netBoxVm(netBoxObject):
    def __init__(self, vm_name: str = '', **kwargs):
        super().__init__()
        self.__put_result = None
        self.__post_result = None
        self.__json = {}
        self.exists = False
        self.authorization = f'Token {self.api_key}'
        self.api = f'{self.url}/api/virtualization/virtual-machines/'
        self.auth = {'Authorization': f'{self.authorization}'}
        self.response = requests.get(url = self.url, headers = {'Authorization': f'{self.authorization}'})
        if self.response.return_code != (200 or 201):
            print(f'Query to {self.url} was unsuccessful for VM {vm_name}\n Continuing with defining new VM of name: {vm_name}')
        else:
            self.__json = json.loads(self.response.content)

    def __repr__(self):
        return self.__json

    @property 
    def json(self):
        return self.__json
    
    @property
    def put_result(self):
        return self.__put_result

    @property
    def post_result(self):
        return self.__post_result

    @property
    def get_result(self):
        return self.__get_result

    def create_vm(self):
        self.__post_result = requests.post(url = self.api, data = json.dumps(self.__json), headers = {'Authorization': f'{self.authorization}', 'Content-Type': 'application/json' })

    def update_vm(self):
        self.__put_result = requests.put(url = self.api, data = json.dumps(self.__json), headers = {'Authorization': f'{self.authorization}', 'Content-Type': 'application/json' })

    def get_vm(self):
        self.__get_result = requests.get(url = f'{self.api}?name={self.name}', data = json.dumps(self.__json), headers = {'Authorization': f'{self.authorization}', 'Content-Type': 'application/json' })
        if self.__get_result:
            self.__json = json.loads(self.__get_result.content)

    def define_self(self, cluster = {}, rep = None, **kwargs):
        # For yeeting in values from another source, probably should be modified to take in a data structure as well, or overloaded for the same functionality 
        if cluster:
            self.__json['cluster'] = cluster
        if rep:
            self.__json = rep
        for key, value in kwargs.items():
            self.__json[key] = value

    def query_vms(nb_url, api_key, vm_name = None):
        nb_vms = f'{nb_url}/virtualization/virtual-machines'
        if vm_name:
            nb_vms = f'{nb_vms}?name={vm_name}'
        headers = {'Authorization': f'Token {api_key}', 'Accepts': 'application/json'}
        result = requests.get(url = nb_vms, headers = headers)
        return result

    def add_vm(self, nb_url, api_key, cluster, vm_name, local_context):
        payload = {
            "name": vm_name,
            "cluster": cluster,
            "vcpus": 1,
            "local_context": json.dumps(local_context),
            "tags": [],
            "comments": ''
        }
        headers = {'content-type': 'application/json', 'Authorization': f'{self.authorization}'}
        result = requests.post(url = f'{nb_url}/virtualization/virtual-machines/', headers = headers, data = payload)
        return result

    def update_vm(self, nb_url, api_key, cluster, vm_name, local_context):
        payload = {
            "name": vm_name,
            "cluster": cluster,
            "vcpus": 1,
            "local_config_context": json.dumps(local_context),
            "tags": [],
            "comments": ''
        }
        headers = {'content-type': 'application/json', 'Authorization': f'{self.authorization}'}
        result = requests.put(url = f'{nb_url}/virtualization/virtual-machines/', headers = headers, data = payload)
        return result

def get_cluster_by_name(auth, url: str = 'netbox.local', cluster_name = 'jupiter'):
    query_url = f'{url}/api/virtualization/clusters'
    if not re.search('^Bearer .*$', auth):
        auth = f'Bearer {auth}'
    headers = dict()
    headers['Authorization'] = auth 
    headers['Content-Type'] = 'application/json'
    cluster_list = requests.get(url = query_url, auth = auth)
    for cluster in cluster_list.json()['results']:
        if cluster['name'] == cluster_name:
            return cluster
    return None

def get_tenant_by_name(auth, url: str = 'netbox.local', tenant_name = 'jupiter'):
    query_url = f'{url}/api/devices/tenants'
    if not re.search('^Bearer .*$', auth):
        auth = f'Bearer {auth}'
    headers = dict()
    headers['Authorization'] = auth 
    headers['Content-Type'] = 'application/json'
    tenant_list = requests.get(url = query_url, auth = auth)
    for tenant in tenant_list.json()['results']:
        if tenant['name'] == tenant_name:
            return tenant
    return None

def get_resource_by_name(name: str, auth: str, url: str = 'netbox.local', kind: str = 'virtual-machine') -> requests.Response:
    query_url = f'{url}/api'
    resource_types = dict()
    resource_types['device_list'] = ['tenant', 'tenant_group']
    resource_types['virtualization_list'] = ['virtual-machine', 'cluster', 'cluster_group']
    for resource_type in resource_types.keys():
        api_middle = resource_type.split('_')[0]
        for endpoint in resource_types[resource_type]:
            if endpoint == kind:
                query_url += f'{query_url}/{api_middle}/{endpoint}'
                break
    if not re.search('^Bearer .*$', auth):
        auth = f'Bearer {auth}'
    headers = dict()
    headers['Authorization'] = auth 
    headers['Content-Type'] = 'application/json'
    resource_list = requests.get(url = query_url, auth = auth)
    for resource in resource_list.json()['results']:
        if resource['name'] == name:
            return resource
    return None


In [18]:
class myclass:
    def __init__(self):
        self.__name = 'john'
    
    @property
    def name(self):
        return self.__name

class moreclass(myclass):
    def __init__(self):
        super().__init__()
        self.job = 'lol'

me = moreclass()
me2 = myclass()
print(me2._myclass__name)
print(me.name)
print(me.job)

john
john
lol
